In [ ]:
!pip install skorch #biblioteca que uni o scikit-learn e o pytorch, fazendo o treinamento, ajuste e otimização do nosso modelo

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#garantindo sempre os mesmo resultados
np.random.seed(123)
torch.manual_seed(123)

#definindo nossa bd
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[: , 4].values

In [ ]:
#transformamos as classes de variaveis categoricas para variaveis numéricas
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [ ]:
#verificando se a transformção ocorreu
np.unique(classe)

In [ ]:
previsores = previsores.astype('float32') #definindo o tipo dos previsores como float pois temos tamanhos com virgula
classe = classe.astype('int64') #definindo o tipo da classe como int pois temos somente os valores 0,1,2

In [ ]:
#CONSTRUINDO O MODELO
class Classificador_torch(nn.Module):
  def __init__(self):
    #definindo as camadas densas e as camadas de ativação
    super().__init__()
    self.dense0 = nn.Linear(4, 16)
    self.activation0 = nn.ReLU()
    self.dense1 = nn.Linear(16, 16)
    self.activation1 = nn.ReLU()
    self.dense2 = nn.Linear(16, 3)
  #definindo a conexão entre as camadas
  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    return X

In [ ]:
#definindo o numero de épocas, a taxa de aprendizado, o otimizador e o batch size
classificador_sklearn = NeuralNetClassifier(module = Classificador_torch, 
                                            criterion = torch.nn.CrossEntropyLoss, 
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 1000,
                                            batch_size = 10,
                                            train_split = False)

In [ ]:
#FAZENDO A VALIDAÇÃO CRUZADA
resultados = cross_val_score(classificador_sklearn, previsores, classe, cv = 5, 
                             scoring = 'accuracy')

In [ ]:
#extraindo a média e o desvio padrao dos resultados
media = resultados.mean()
desvio = resultados.std()

In [ ]:
media, desvio

In [ ]:
#printando a porcentagem de acerto de cada classe
resultados